NOTE:
-----

Please run the below cells first before proceeding- you'll need them soon!

In [1]:
%load_ext sql
%sql sqlite://

/Library/Python/2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Library/Python/2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


'Connected: None@None'

In [3]:
%%sql
DROP TABLE IF EXISTS franchise;
CREATE TABLE franchise (name TEXT, db_type TEXT);
INSERT INTO franchise VALUES ('Bobs Bagels', 'NoSQL');
INSERT INTO franchise VALUES ('eBagel', 'NoSQL');
INSERT INTO franchise VALUES ('BAGEL CORP', 'MySQL');

DROP TABLE IF EXISTS store;
CREATE TABLE store (franchise TEXT, location TEXT);
INSERT INTO store VALUES ('Bobs Bagels', 'NYC');
INSERT INTO store VALUES ('eBagel', 'PA');
INSERT INTO store VALUES ('BAGEL CORP', 'Chicago');
INSERT INTO store VALUES ('BAGEL CORP', 'NYC');
INSERT INTO store VALUES ('BAGEL CORP', 'PA');

DROP TABLE IF EXISTS bagel;
CREATE TABLE bagel (name TEXT, price MONEY, made_by TEXT);
INSERT INTO bagel VALUES ('Plain with shmear', 1.99, 'Bobs Bagels');
INSERT INTO bagel VALUES ('Egg with shmear', 2.39, 'Bobs Bagels');
INSERT INTO bagel VALUES ('eBagel Drinkable Bagel', 27.99, 'eBagel');
INSERT INTO bagel VALUES ('eBagel Expansion Pack', 1.99, 'eBagel');
INSERT INTO bagel VALUES ('Plain with shmear', 0.99, 'BAGEL CORP');
INSERT INTO bagel VALUES ('Organic Flax-seed bagel chips', 0.99, 'BAGEL CORP');

DROP TABLE IF EXISTS purchase;
-- Note that date is an int here just to simplify things
CREATE TABLE purchase (bagel_name TEXT, franchise TEXT, date INT, quantity INT, purchaser_age INT);
INSERT INTO purchase VALUES ('Plain with shmear', 'Bobs Bagels', 1, 12, 28);
INSERT INTO purchase VALUES ('Egg with shmear', 'Bobs Bagels', 2, 6, 47);
INSERT INTO purchase VALUES ('Plain with shmear', 'BAGEL CORP', 2, 12, 24);
INSERT INTO purchase VALUES ('Plain with shmear', 'BAGEL CORP', 3, 1, 17);
INSERT INTO purchase VALUES ('eBagel Expansion Pack', 'eBagel', 1, 137, 5);
INSERT INTO purchase VALUES ('Plain with shmear', 'Bobs Bagels', 4, 24, NULL);

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Recall that the database we just loaded has one table, `precipitation`, having the following schema:

> * `state_code`
> * `station_id`
> * `year`
> * `month`
> * `day`
> * `hour`
> * `precipitation`
> * `flag_1`
> * `flag_2`

Each tuple in this table describes one hour of rainfall (`precipitation`- in hundredths of an inch) at one station (`station_id`) in one state (`state_code`).  Note that tuples with `hour=25` record the total rainfall for that day, and that we can ignore the values of attributes `flag_1` and `flag_2` for now.

Activity 3-2
------------
Aggregation operators, GROUP BY

Exercise #1
-----------

Consider the following set of products and purchases:

In [5]:
%sql SELECT * FROM bagel;

Done.


name,price,made_by
Plain with shmear,1.99,Bobs Bagels
Egg with shmear,2.39,Bobs Bagels
eBagel Drinkable Bagel,27.99,eBagel
eBagel Expansion Pack,1.99,eBagel
Plain with shmear,0.99,BAGEL CORP
Organic Flax-seed bagel chips,0.99,BAGEL CORP


In [6]:
%sql SELECT * FROM purchase;

Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17
eBagel Expansion Pack,eBagel,1,137,5
Plain with shmear,Bobs Bagels,4,24,None


Can you write a query to get the _total revenue_ for each bagel type **which had an average purchaser age over 18**?  Type your query below:

In [10]:
# Result you should get (do not re-execute this cell!)
_

name,Revenue
Egg with shmear,14.34
Plain with shmear,84.51


Exercise #2
-----------

Let's load the precipitation database and do some testing of GROUP BY vs. nested querying!

In [11]:
%sql sqlite:///precipitation.db

u'Connected: None@precipitation.db'

In [12]:
%sql SELECT code FROM states WHERE abbrev = 'CA';

Done.


code
4


In [13]:
%sql SELECT COUNT(DISTINCT station_id) AS "# of CA Weather Stations" FROM precipitation WHERE state_code=4;

Done.


# of CA Weather Stations
165


We want to get station_ids of stations in CA (state_code = 4) which have average _end of day_ (hour = 25) precipitations > 75.  Try doing this first as a nested query:

Now, try re-writing as a GROUP BY:

In [16]:
# Result you should get in both cases (do not re-execute this cell!)
_

station_id
88302
250002
335701
357302
488301


Now time it by using `%time` followed by single-line versions of your queries above (clunky, but will work) to see how they compare!

**Note:** Yes, currently the answers are filled in below for convenience... but you should still try getting them on your own above!

In [17]:
%time %sql SELECT DISTINCT p.station_id FROM precipitation p WHERE p.state_code = 4 AND p.hour = 25 AND (SELECT AVG(precipitation) FROM precipitation WHERE station_id = p.station_id AND hour = 25) > 75;

Done.
CPU times: user 4.49 s, sys: 5.7 ms, total: 4.49 s
Wall time: 4.49 s


station_id
88302
250002
335701
357302
488301


In [18]:
%time %sql SELECT p.station_id FROM precipitation p WHERE p.state_code = 4 AND p.hour = 25 GROUP BY p.station_id HAVING AVG(p.precipitation) > 75;

Done.
CPU times: user 8.58 ms, sys: 1.1 ms, total: 9.68 ms
Wall time: 8.58 ms


station_id
88302
250002
335701
357302
488301
